# Approximate Bayesian Ensembling - Regression

In [1]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.enable_eager_execution()
import matplotlib.pyplot as plt
import numpy as np


2024-11-22 22:41:08.939951: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-22 22:41:09.210341: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-22 22:41:09.210389: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-22 22:41:09.212002: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 22:41:09.326125: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-22 22:41:09.327420: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Instructions for updating:
non-resource variables are not supported in the long term
